# Objective

We've collected data across different variables from our customers. Our job is to look at this data to see if any patterns emerge about a client's likelihood to pay back a loan given these variables. 

Specifically, we were asked to investigate:
- Does a customer's marital status impact whether or not they will default on a loan?
- Does the number of children a customer has effect whether or not they will default on a loan?

In [1]:
#import libraries
import pandas as pd
import numpy as np

In [2]:
#read dataset
try:
    credit_df = pd.read_csv('credit_scoring_eng.csv')
except:
    credit_df = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [3]:
#inspect the data
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


# Preprocessing

## 1. Null Values

There are null values in the 'days_employed' column and the 'total_income' column. They are also the same number of null values which is interesting...

In [4]:
#a quick look at the null values

credit_df[credit_df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


It looks like the null values in the 'days_employed' column are also null values in the 'total_income' column. Let's run a check to see.

In [5]:
credit_df[credit_df['days_employed'].isna()].shape 

#looking at the shape of null value dataframes

(2174, 12)

In [7]:
credit_df[credit_df['total_income'].isna()].shape 

#looking at the shape of null value dataframes

(2174, 12)

In [8]:
credit_df.loc[credit_df['days_employed'].isna()&credit_df['total_income'].isna()].shape 

#looking at the shape of null value dataframes

(2174, 12)

Above we looked at the shape of a dataframe of just the null values in the 'days_employed' column, a dataframe of just the null values in the 'total_income' column, and a dataframe of just the rows of data that have BOTH these columns null. What we got was that all 3 of these have the same number of rows. Which confirms that any row with a null value in 'days_employed' also has a null value in 'total_income'. (If any row didn't have both null, there would be a mismatch in the number of rows).

In [9]:
credit_df.loc[credit_df['days_employed'].isna()&credit_df['total_income'].isna()].shape[0]/credit_df.shape[0]

#dividing the number of rows with null data, by the length of dataset to give what percentage of the data has null values

0.10099883855981417

Looks like the null values account for about 10% of the data, let's see if there are any patterns between the null data and the other columns. Since we are investigating on having children and family status, lets look at those first.

In [10]:
null_data = credit_df.loc[credit_df['days_employed'].isna()&credit_df['total_income'].isna()]
null_data['children'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

In [12]:
credit_df['children'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Aside from the strange values (20 kids and -1 kids, which we will deal with later), it seems the distribution of this column is similar between the rows of data that have null values and the rows of data that do not have null values.

In [13]:
null_data['family_status'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64

In [14]:
credit_df['family_status'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

Similarly, the distribution in 'family_status' (and therefore 'family_status_id' since they give the same information) is similar amongst the data with null values, and the data without null values.

In [15]:
null_data['dob_years'].describe()

#comparing distribution of null value dataframe to entire dataframe
#we use describe here since dob_years is a numerical column

count    2174.000000
mean       43.632015
std        12.531481
min         0.000000
25%        34.000000
50%        43.000000
75%        54.000000
max        73.000000
Name: dob_years, dtype: float64

In [16]:
credit_df['dob_years'].describe()

#comparing distribution of null value dataframe to entire dataframe
#we use describe here since dob_years is a numerical column

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [17]:
null_data['gender'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

F    1484
M     690
Name: gender, dtype: int64

In [18]:
credit_df['gender'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [20]:
null_data['debt'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

0    2004
1     170
Name: debt, dtype: int64

In [21]:
credit_df['debt'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

0    19784
1     1741
Name: debt, dtype: int64

In [22]:
null_data['education_id'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64

In [23]:
credit_df['education_id'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [24]:
null_data['income_type'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

In [25]:
credit_df['income_type'].value_counts()

#comparing distribution of null value dataframe to entire dataframe

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

Above, we looked at the distribution of every other column in both a dataset with only the null values and compared it to the entire dataset in order to try to establish a pattern about why these values were missing.

At first, we thought maybe the reasoning behind the null values is that the customers were unemployed, or weren't currently making an income, but upon seeing the distribution of the 'income_type' column, we saw that it was very similar to the distribution of the entire data set. Aside from having null values, this 10% of the data is indistinguishable from the rest of the data set, and so we cannot establish a pattern. These datapoints however still contain other data, including the variables we want to focus on; children and marital status, in relation to the debt column; so we do not want to drop them. Let's look at the columns where these null values exist, in order to determine what the best way to fill these values are. We will also take the time to address the strange values we saw while checking the distribution of the other columns.

## 2. Strange Values in the Data

### A) days_employed column and missing values

We are investigating this column first in order to fill the null values in the data set which exist in this column.

In [26]:
credit_df['days_employed'].describe()

#using describe to check the distribution of the 'days_employed' column because it is numerical

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

It looks like the majority of these values are negative, this was probably caused by an error in the gathering of the dates It is likely that the true value is just the absolute value of these negative numbers. But the average is still positive, which tells us that there are positive values here that are very large in magnitude(outliers). Now that we recognize that there are outliers, let's replace our null values with the median, after which, we will investigate the negative values and address the outliers.

In [27]:
credit_df['days_employed'].fillna(credit_df['days_employed'].median(),inplace=True)
credit_df['days_employed'].describe()

#filled null with median, and checked to see if changes persisted

count     21525.000000
mean      56557.335698
std      134922.319298
min      -18388.949901
25%       -2518.168900
50%       -1203.369529
75%        -385.106616
max      401755.400475
Name: days_employed, dtype: float64

Next we will investigate the large positive outliers.

In [28]:
credit_df[credit_df['days_employed'] > 0]['days_employed'].min()

#code to get the smallest value above 0 in the days employed column

328728.72060451825

Looks like the smallest value over 0 is about 329729 days...which is over 900years! Well beyond the lifespan of a human. these are all major outliers and are probably a mistake.

In [29]:
credit_df[credit_df['days_employed'] > 0].shape[0]/credit_df.shape[0]

#dividing number of values above 0, by length of data frame to get what percentage of this dataframe have days employed
#values that are illogical

0.16004645760743322

These errors account for about 16% of the data, that's a huge portion! Being that these are all huge outliers - let's replace these values with the median. We will also do this for the null values in this column.

In [30]:
def repWithMedian(number):
    """
    Replaces a number that is greater than 0 with the median value of the 'days_employed' column
    """
    if number > 0:
        number = credit_df['days_employed'].median()
    return number

#function to help me fix days_employed column

In [31]:
credit_df['days_employed'] = credit_df['days_employed'].apply(repWithMedian)

#applying the function

In [32]:
credit_df['days_employed'].describe()

#checking to make sure function worked and changes persisted

count    21525.000000
mean     -2052.906146
std       2044.116204
min     -18388.949901
25%      -2518.168900
50%      -1203.369529
75%      -1025.608174
max        -24.141633
Name: days_employed, dtype: float64

Now that we've replaced them with more realistic values, let's make them all positive.

In [33]:
credit_df['days_employed'] = credit_df['days_employed'].apply(abs)
credit_df['days_employed'].describe()

#changing the values from negative to positive.

count    21525.000000
mean      2052.906146
std       2044.116204
min         24.141633
25%       1025.608174
50%       1203.369529
75%       2518.168900
max      18388.949901
Name: days_employed, dtype: float64

Let's move on to look at another column

### B) total_income column and missing values

In [34]:
credit_df['total_income'].describe()

#using describe to check the distribution of the 'days_employed' column because it is numerical

count     19351.000000
mean      26787.568355
std       16475.450632
min        3306.762000
25%       16488.504500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64

Since this is the other column with missing values, we want to replace them with the median. However we want to account for the difference in occupation for different people, since people in different occupations tend to make vastly different incomes.

In [35]:
credit_df['income_type'].value_counts()

#seeing the different types of income

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [36]:
credit_df.groupby('income_type')['total_income'].median()

#checking the median of each income_type category

income_type
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

In [37]:
median_incomes = credit_df.groupby(by='income_type')['total_income'].median()

#for easier reference to the different medians in the next lines of code

In [38]:
inc_type_list = ['business', 'civil servant', 'employee', 'entrepreneur', 
                 'paternity / maternity leave', 'retiree', 'student', 'unemployed']

#making a list of each income_type in the same order as the 'median_incomes' variable

In [40]:
for i, inc_type in enumerate(inc_type_list):
    credit_df.loc[credit_df['income_type']==inc_type,'total_income'] = credit_df.loc[credit_df['income_type']==inc_type,'total_income'].fillna(median_incomes[i])

#for loop which iterates over the list of income types, and replaces fills null 
#values of that income type with the respective median of that income type

In [41]:
credit_df['total_income'].isna().sum()

#checking to make sure that there are no null values

0

Now that we've dealt with the null values, let's look at the odd values in the other columns

### C) gender column and strange values

In [42]:
credit_df['gender'].value_counts()
#checking distribution of gender column

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

We have a weird 'XNA' value, since its only occurring once, we will drop this error.

In [43]:
credit_df['gender'].replace('XNA',np.nan,inplace=True)
credit_df['gender'].value_counts()

#dropping rows where value for gender is XNA, and then checking to ensure changes persisted

F    14236
M     7288
Name: gender, dtype: int64

### D) dob_years column and strange values

In [44]:
credit_df['dob_years'].value_counts()

#checking distribution of dob_years column

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Looks like there are about 101 entries, saying that theyre 0 years old. How many others claim that they are under 18?

In [45]:
credit_df[credit_df['dob_years']<18].shape[0]

#checking how many of these clients are under 18 in order to see where there are errors

101

Only those 101 who are age "0". Since there aren't many outliers in this column, let's replace these age 0s with the mean. Since the mean won't be an integer, Let's wrap it in int()

In [46]:
credit_df['dob_years'].replace(0,int(credit_df['dob_years'].mean()),inplace=True)

#replacing these 0 values with the mean value, and rounding it with the int() function

In [48]:
credit_df['dob_years'].value_counts()

#checking if the changes persisted

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Let's move on to another column.

### E) education column and strange values

In [49]:
credit_df['education'].value_counts()

#checking the distribution of the education column

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

There's a lot of issues with capitalization here, lets fix them.

In [50]:
education_corrections = {
    'SECONDARY EDUCATION':'secondary education',
    'Secondary Education':'secondary education',
    "BACHELOR'S DEGREE":"bachelor's degree",
    "Bachelor's Degree":"bachelor's degree",
    'Some College':'some college',
    'SOME COLLEGE':'some college',
    'PRIMARY EDUCATION':'primary education',
    'Primary Education':'primary education',
    'Graduate Degree':'graduate degree',
    'GRADUATE DEGREE':'graduate degree'}

credit_df['education'].replace(education_corrections, inplace=True)

credit_df['education'].value_counts()
#replacing the education column to standardized categories, and checking if the changes persisted

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

Much better, lets move on to the next column.

### F) purpose column and strange values

In [51]:
credit_df['purpose'].value_counts()

#seeing distribution of purpose column

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

I want to categorize this data better in to just 4 categories, real estate, wedding, car, and education

In [52]:
def categorize(x):
    if "wedding" in x:
        return "wedding"
    elif "real estate" in x:
        return "real estate"
    elif "house" in x:
        return "real estate"
    elif "property" in x:
        return "real estate"
    elif "housing" in x:
        return "real estate"
    elif "car" in x:
        return "car"
    elif "university" in x:
        return "education"
    elif "education" in x:
        return "education"
    elif "educated" in x:
        return "education"
    else:
        return "None"
    
#function to help categorize the purpose column

In [53]:
credit_df['purpose_cat'] = credit_df['purpose'].apply(categorize)

#applying the function

In [54]:
credit_df['purpose_cat'].value_counts()

#checking to make sure the function works

real estate    10840
car             4315
education       4022
wedding         2348
Name: purpose_cat, dtype: int64

Much better, now onto the next column

### G) children column and strange values

In [55]:
credit_df['children'].value_counts()

#checking the distribution of the children column

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [56]:
(76+47)/credit_df.shape[0]

#checking what percentage of the data are made up by the error values of 20 and -1

0.005714285714285714

Looks like there are some errors here. I'm sure -1 children is a mistake that was supposed to be 1 child, and 20 children was a mistake that is meant to be 2 children. It is highly unlikely that there's such a huge jump to 20 children, and no one has in between 5 and 20 children. Both these values are mistakes.

Let's replace them with appropriate values.

In [57]:
credit_df['children'].replace({
    20:2,
    -1:1}, inplace=True)

#replacing the error values with other values

In [58]:
credit_df['children'].value_counts()

#checking to ensure changes persisted

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### H) Miscellaneous Columns with no strange values

In [59]:
credit_df['education_id'].value_counts()

#checking distribution for strange values

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [60]:
credit_df['family_status'].value_counts()

#checking distribution for strange values

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [61]:
credit_df['family_status_id'].value_counts()

#checking distribution for strange values

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [62]:
credit_df['income_type'].value_counts()

#checking distribution for strange values

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [63]:
credit_df['debt'].value_counts()

#checking distribution for strange values

0    19784
1     1741
Name: debt, dtype: int64

Nothing wrong or suspicious in any of the above columns, let's move on.

## Duplicates

Last thing before moving on, check for duplicates!

In [64]:
credit_df.duplicated().sum()

#check for duplicates

71

In [66]:
credit_df[credit_df.duplicated()]

#looking at the duplicates closer

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat
2849,0,1203.369529,41,secondary education,1,married,0,F,employee,0,22815.1035,purchase of the house for my family,real estate
3290,0,1203.369529,58,secondary education,1,civil partnership,1,F,retiree,0,18962.3180,to have a wedding,wedding
4182,1,1203.369529,34,bachelor's degree,0,civil partnership,1,F,employee,0,22815.1035,wedding ceremony,wedding
4851,0,1203.369529,60,secondary education,1,civil partnership,1,F,retiree,0,18962.3180,wedding ceremony,wedding
5557,0,1203.369529,58,secondary education,1,civil partnership,1,F,retiree,0,18962.3180,to have a wedding,wedding
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,1203.369529,64,secondary education,1,married,0,F,retiree,0,18962.3180,supplementary education,education
21032,0,1203.369529,60,secondary education,1,married,0,F,retiree,0,18962.3180,to become educated,education
21132,0,1203.369529,47,secondary education,1,married,0,F,employee,0,22815.1035,housing renovation,real estate
21281,1,1203.369529,30,bachelor's degree,0,married,0,F,employee,0,22815.1035,buy commercial real estate,real estate


Looks like we have some duplicate rows. Thinking as to why some of these might be duplicates, these are all rows where I filled total income and days employed with the median. But what are the chances that two people have the same number of children, same age, education level, family status, gender, income type, and debt values? It should be extremely rare, but 71 of 21525 is really rare. So this may be two people who just happen to lead very similar lives. The one difference is, that the purpose column looks like they had individuals type, and what are the chances that people type in the exact same response here? I'd say very very rare, and this is one column we did not alter in any way. Because of this, I believe these are actual duplicate entries, instead of just two people who live similar lives and will be dropping the duplicate rows.

In [67]:
credit_df.drop_duplicates(inplace=True)

#dropping duplicates

In [68]:
credit_df.duplicated().sum()

#checking to see that all duplicates were taken care of

0

## What we've done so far:

Looks like there are no duplicates left, so the data should be ready for analysis! Let's quickly go over what was done:

- Found null data in 'days_employed' and 'total_income' columns which accounted for about 10% of the total data, after coming to the conclusion that there was no pattern here, we filled these values with appropriate median values from the dataset
- Saw that 16% of the values in 'days_employed' could not be real as they were beyond the lifespan of humans! Since these were outliers, we replaced these values with the median.
- Saw that the values in 'days_employed' were all negative, and applied the abs() function to get the absolute values to make these values positive.
- Saw that the 'gender' column had a value 'XNA' and so we dropped this one row
- Found 101 entrees in the 'dob_years' column where users claimed to be 0 years old. This is clearly a mistake, no one under 18 can apply for a loan. We replaced these values with the rounded mean age, since there were no outliers.
- Found that the categorical values in the 'education' has inconsistent cases, so we made them all lowercase.
- Categorized the 'purpose' column into 4 categories, wedding, real estate, car, and education into new column 'purpose_cat'
- The 'children' column 76 entries claiming to have 20 children, and 47 claiming to have -1 children, these were interpreted as mistakes and 2 was put in for 20, and 1 was put in for -1.
- All other columns did not seem suspicious or wrong at all.
- Dropped the rows that seemed to be duplicates.

# Categorizing the Data

It's time to categorize the data!
The questions we want to answer:
- Is there a connection between having kids and repaying loans on time?

In order to answer this question, we will compare the loan default rate to see if there's a correlation with how many children the client has.

- Is there a connection between marital status and repaying loans on time?

In order to answer this question, we will group the data by marital status

- Is there a connection between income level and repaying loans on time?

In order to answer this question, we will group the data by income level

- How do different loan purposes affect on-time loan repayment?

In order to answer this question, we will group the purposes

Let's get started.

### Children and Defaulting on Loans

In [70]:
credit_df.pivot_table(index='children',values='debt',aggfunc='sum')/credit_df.pivot_table(index='children',values='debt',aggfunc='count')*100

#pivot table with percentages on who has defaulted on loans compared to number of children they have

,debt
children,
0,7.543822
1,9.165808
2,9.492481
3,8.181818
4,9.756098
5,0.000000


### Conclusion
When looking at the data in the pivot table, it is clear that clients with no children have the least likelihood of defaulting on a loan. There doesn't seem to be a pattern on if the number of children effects the loan repayment rate, as the percentage for clients with 1 child is 9.2%, clients with 2 children at 9.5%, 3 children at 8.2%, and 4 children at 9.8%, whereas 5 children default rate is 0%. this last one is likely due to how few clients actually have 5 children.

### Marital Status and Defaulting on Loans

In [72]:
credit_df['family_status'].value_counts()

#seeing distribution of family_status column

married              12339
civil partnership     4151
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64

We will separate these by those who are in a relationship (married/civil partnership) those who were in a relationship (divorced/widowed) and those who have never been in a relationship (unmarried).

In [73]:
credit_df.pivot_table(index='family_status',values='debt',aggfunc='sum')/credit_df.pivot_table(index='family_status',values='debt',aggfunc='count')*100

#pivot table with percentages on who has defaulted on loans compared to their marital status

,debt
family_status,
civil partnership,9.347145
divorced,7.112971
married,7.545182
unmarried,9.750890
widow / widower,6.569343


### Conclusion
It looks like those who are unmarried have the highest rate of loan defaulting at 9.8%. The next closest is civil partnerships at 9.3%, and after that all other marital statuses have below a 7.6% loan default rate.
Unmarried clients default on their loans more often than other marital statuses, including civil partnerships, divorces, widows, and married clients.

### Total_income and defaulting on loans

In [74]:
credit_df['total_income'].describe()

#seeing the distribution of the total income

count     21454.000000
mean      26451.212929
std       15709.968189
min        3306.762000
25%       17219.817250
50%       22815.103500
75%       31331.348000
max      362496.645000
Name: total_income, dtype: float64

We will separate the total income level be quantile. There will be a Low, Low Middle, High Middle, and High income groups

In [75]:
first_quant = credit_df['total_income'].describe()[4]
median = credit_df['total_income'].describe()[5]
third_quant = credit_df['total_income'].describe()[6]

In [77]:
#row function to make a column based on income levels

def income_level(x):
    if x <= first_quant:
        return 'low'
    elif first_quant < x <= median:
        return 'low middle'
    elif median < x <= third_quant:
        return 'high middle'
    elif x > third_quant:
        return 'high'

credit_df['income_level'] = credit_df['total_income'].apply(income_level)

#applying the function by row

In [78]:
credit_df.pivot_table(index='income_level',values='debt',aggfunc='sum')/credit_df.pivot_table(index='income_level',values='debt',aggfunc='count')*100

#pivot table with percentages on who has defaulted on loans compared to their income level

,debt
income_level,
high,7.140194
high middle,8.538212
low,7.960477
low middle,8.815477


### Conclusion
Looks like middle income earners tend to default on their loans more often than low income, or high income earners. This can be because low income earners most likely have trouble qualifying for loans to begin with, and high income earners make enough income that they can repay the loan. This is shown percentage-wise because low-middle and high-middle income classes default on loans at a rate of 8.8% and 8.5% respectively. Which is higher than both low income (8.0%) and high income (7.1%) percentages.

### Loan purpose and defaulting on loans

We have already separated the 'purpose' column into categories in the 'purpose_cat' column

In [79]:
credit_df['purpose_cat'].value_counts()

#checking distribution of purpose category column

real estate    10811
car             4306
education       4013
wedding         2324
Name: purpose_cat, dtype: int64

In [80]:
credit_df.pivot_table(index='purpose_cat',values='debt',aggfunc='sum')/credit_df.pivot_table(index='purpose_cat',values='debt',aggfunc='count')*100

#pivot table with percentages on who has defaulted on loans compared to their purpose for applying for the loan

,debt
purpose_cat,
car,9.359034
education,9.220035
real estate,7.233373
wedding,8.003442


### Conlcusion
Those who take out a loan with the purpose of spending the money on a car have the highest rate of defaulting on their loan at 9.4%, followed by those who take out a loan for education at 9.2%, then by those who take out a loan for a wedding at 8.0% and finally by those who take out a loan for real estate at 7.2%

### Final Thoughts

According to our data:
- Clients with children are more likely to default on their loans than clients without children.
- Clients who are unmarried are more likely to default on their loans than clients in any other marital situation.
- Clients who make between 17,000 and 22,000 are more likely to default on their loans than clients in other income ranges.
- Clients who are taking out a loan for a car, are more likely to default on their loans than clients seeking loans for other reasons.

The best loan candidate would:
- Have no children
- Be widowed
- Make over 31,000 salary
- Is looking to invest in real estate with the loan

The worst loan candidate would:
- Have 4 children
- Be unmarried
- Make between 17,000 and 22,000
- Is looking to use the loan for a car